# KoELECTRA 모델로 fine-tuning 하기

- Task : KLUE-NER
- 담당자: [전명준](https://github.com/oncores) 님
- 최종수정일: 21-09-15
- 본 자료는 가짜연구소 3기 KLUE 로 모델 평가하기 크루 활동으로 작성됨

# 진행 상황
- validation set 3 epoch 기준 
    - entity_macro_f1 = 84.86452184810163
    - character_macro_f1 = 92.31535366001408

- validation loss가 지속적으로 감소되어 early stopping 기준이 되지 않아 중간 체크포인트 저장

# fine-tuning 과정
- 사용 모델 : KoELECTRA
- 주요 파라미터
    - max length : 256
    - learning_rate 3e-5
    - num_train_epochs 5
    - train_batch_size 16

## initialize

In [ ]:
cd ./drive/MyDrive/Colab Notebooks/klue/KLUE-baseline-main

/content/drive/MyDrive/Colab Notebooks/klue/KLUE-baseline-main


In [ ]:
ls

data/           model/          README_ko.md          run_all.sh
klue_baseline/  mypy.ini        README.md             run_klue.py
LICENSE.md      ner/            requirements-dev.txt  setup.cfg
Makefile        pyproject.toml  requirements.txt      wos/


In [ ]:
!pip install -r requirements.txt

Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     |████████████████████████████████| 1.3 MB 4.2 MB/s 
     |████████████████████████████████| 665 kB 43.4 MB/s 
     |████████████████████████████████| 108 kB 44.2 MB/s 
     |████████████████████████████████| 22.3 MB 19.7 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 2.9 MB 40.8 MB/s 
     |████████████████████████████████| 1.1 MB 38.5 MB/s 
     |████████████████████████████████| 895 kB 58.2 MB/s 
     |████████████████████████████████| 636 kB 53.7 MB/s 
     |████████████████████████████████| 829 kB 54.0 MB/s 
  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl size=10186 sha256=65c0837cbc1d8726e7cf6708d08c98ce5fc061e1e795f270af90ee9cab72a7a0
  Stored in directory: /root/.cache/pip/wheels/3a/0d/38/01a9bc6e20dcfaf0a6a7b552d03137558ba1c38aea47644682
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=49

In [ ]:
!pip install torchtext==0.8.0 torch==1.7.1 pytorch-lightning==1.2.2

     |████████████████████████████████| 776.8 MB 17 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.7.1 which is incompatible.


## train

In [ ]:
!python run_klue.py \
train \
--task klue-ner \
--output_dir ./ner_output \
--data_dir ./ner \
--model_name_or_path monologg/koelectra-base-v3-discriminator \
--learning_rate 3e-5 \
--num_train_epochs 5 \
--train_batch_size 16 \
--eval_batch_size 16 \
--max_seq_length 256 \
--gradient_accumulation_steps 1 \
--warmup_ratio 0.1 \
--weight_decay 0.01 \
--max_grad_norm 1.0 \
--patience 100000 \
--metric_key character_macro_f1 \
--gpus 0 \
# --parallel_decoding \
# --truncate \

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: MetricBase will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
08/22/2021 06:56:38 - INFO - __main__ - Arguments:
08/22/2021 06:56:38 - INFO - __main__ - 	                command : train
08/22/2021 06:56:38 - INFO - __main__ - 	                   task : klue-ner
08/22/2021 06:56:38 - INFO - __main__ - 	             output_dir : ./ner_output
08/22/2021 06:56:38 - INFO - __main__ - 	                   gpus : [0]
08/22/2021 06:56:38 - INFO - __main__ - 	                   fp16 : False
08/22/2021 06:56:38 - INFO - __main__ - 	   num_sanity_val_steps : 2
08/22/2021 06:56:38 - INFO - __main__ - 	              tpu_cores : None
08/22/2021 06:56:38 - INFO - __main__ - 	      gradient_clip_val : 1.0
08/22/2021 06:56:38 - INFO - __main__ - 	accumulate_grad_batches : 1
08/22/2021 06:56:38 - INFO - __main__ - 	  

## evalutation

In [ ]:
!python run_klue.py \
Evaluate \
--task klue-ner \
--output_dir ./ner/ner_output \
--data_dir ./ner \
--model_name_or_path ./ner/ner_output/klue-ner/version_1/transformers \
--eval_batch_size 128 \

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: MetricBase will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
08/25/2021 13:59:56 - INFO - __main__ - Arguments:
08/25/2021 13:59:56 - INFO - __main__ - 	                command : Evaluate
08/25/2021 13:59:56 - INFO - __main__ - 	                   task : klue-ner
08/25/2021 13:59:56 - INFO - __main__ - 	             output_dir : ./ner/ner_output
08/25/2021 13:59:56 - INFO - __main__ - 	                   gpus : None
08/25/2021 13:59:56 - INFO - __main__ - 	                   fp16 : False
08/25/2021 13:59:56 - INFO - __main__ - 	   num_sanity_val_steps : 2
08/25/2021 13:59:56 - INFO - __main__ - 	              tpu_cores : None
08/25/2021 13:59:56 - INFO - __main__ - 	      gradient_clip_val : 1.0
08/25/2021 13:59:56 - INFO - __main__ - 	accumulate_grad_batches : 1
08/25/2021 13:59:56 - INFO - __main

## inference

In [ ]:
!python run_klue.py \
--model_dir ./ner/ner_output/klue-ner/version_1/transformers \
--eval_batch_size 128 \

data/           Makefile  pyproject.toml        requirements.txt  wos/
inference.py    model/    README_ko.md          run_all.sh
klue_baseline/  mypy.ini  README.md             run_klue.py
LICENSE.md      ner/      requirements-dev.txt  setup.cfg
